# How to Create a US Census Bar Chart Race
## Overview
The beginning of every data science project starts with data and this will be the focus of this project. Usually, this part involves three steps:
 * data acquisition,
 * data cleaning and
 * data visualization.
 
Data visualization can mean anything from simple print statements to fancy plots and animations.

## Data Acquisition
As a data source, we use Wikipedia's census data websites. Since this data is only available on the web, we need to parse the website and extract the useful data. We use the popular [pandas](https://pandas.pydata.org/) package for this task. 

In [1]:
import pandas as pd

We also be using two more packages, `pickle` which allows us to easily save data to disk and `re` which allows us to use [regular expression](https://regexr.com/) syntax. Finally, for nicer table formatting, we import `IPython.display`.

In [2]:
import pickle
import re
from IPython.display import display, HTML

# Define empty Census data dictionary - the keys will be the census years.
Census_raw = {}

### Parsing Wikipedia
Generally, parsing is not recommended to get data from the web as we often have to deal with messy formatting and a lot of cleaning up is needed. However, in this case, Wikipedia's layout of the US census years seems relatively consistent and thus we give parsing a shot. The following code goes through all Wikipedia census websites from [1790](https://en.wikipedia.org/wiki/1790_United_States_Census) to [2010](https://en.wikipedia.org/wiki/2010_United_States_Census) and collects all tables from the pages. We also add the latest census estimate from 2019.

In [3]:
# Uncomment if you like to parse the data again (not recommended!)

# for year in range(1790,2020,10):
#     Census_raw[year]=pd.read_html('https://en.wikipedia.org/wiki/{}_United_States_Census'.format(year))
# Census_raw[2019]=pd.read_html('https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population')

Once, we have parsed the data for the first time, we save it to disk and when we run our notebook the next time, we can skip the parsing step.

In [4]:
if len(Census_raw.keys()):
    with open('rawdata.pickle', 'wb') as handle:
        pickle.dump(Census_raw, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open('rawdata.pickle', 'rb') as handle:
        Census_raw = pickle.load(handle)

## Data Cleaning

All Census pages contain several tables but we are only interested in those tables that contain `State` (or `District` as in [1800](https://en.wikipedia.org/wiki/1800_United_States_Census)) as one of their columns. Most pages also have a seperate statistic on largest cities and we thus ignore all tables that have `City` as one of their columns.

In [5]:
Census = {}
for key in Census_raw.keys():
    Census[key] = [table for table in Census_raw[key] if ('State' in table.columns) or (('District' in table.columns))]
    if len(Census[key]) > 1:
        Census[key] = [table for table in Census[key] if ('City' not in table.columns)][0]
    else:
        Census[key] = Census[key][0]

After these relatively easy manipulations, we need to get our hands dirty and make sure that all tables are of the correct format. Often, it is best to visualize the tables (or at least the first entry) to get a better idea. So let's do that. 

In [6]:
for key in Census.keys():
    print("Displaying values for {}".format(key))
    display(Census[key].head(1))

Displaying values for 1790


,State,"Free white males of 16 years and upward, including heads of families",Free white males under 16 years,"Free white females, including heads of families",All other free persons,Slaves,Enslaved % of state population,Total,% of US population
0,Vermont,22435,22328,40505,255,16[a][7],0.02%,"85,539[b]",2.2%


Displaying values for 1800


,District,"Free white males under age 10,",Free white males age 10–16,Free white males age 16–26,Free white males age 26–45,Free white males over age 45,Free white females under age 10,Free white females age 10–16,Free white females age 16–26,Free white females age 26–45,Free white females over age 45,All other free persons,Slaves,Total
0,New Hampshire,30694,14881,16379,17589,11715,29871,14193,17153,18381,12142,852,8,183858


Displaying values for 1810


...

,Rank,State,Population as of1990 Census[4],Population as of2000 Census[4],Change,Percentchange
0,1,California,29760021,33871648,4111627,13.8%


Displaying values for 2010


,Rank,State,Population as of2000 Census,Population as of2010 Census[51],Change,Percentchange
0,1,California,33871648,37253956,3382308,10.0%


Displaying values for 2019


,"Rank in the fifty states, 2019","Rank in states & territories, 2010",State,"Population estimate, July 1, 2019[5]","Census population, April 1, 2010[6]","Percent change, 2010–2019[note 1]","Absolute change, 2010-2019","Total seats in the U.S. House of Representatives, 2013–2023","Estimated population per electoral vote, 2019[note 2]","Estimated population per House seat, 2019","Census population per House seat, 2010","Percent of the total U.S. population, 2018[note 3]"
0,1.0,1.0,California,39512223,37254523,6.1%,"+2,257,700",53,718404,745514,702885,11.96%


Ok, so it looks like 1790 and 1800 use `Total` instead of `Population` and 1800 also uses `District` instead of `State`. Let's change that! 

In [7]:
Census[1790].rename(columns = {'Total':'Population'}, inplace=True)
Census[1800].rename(columns = {'Total':'Population', 'District':'State'}, inplace=True)

Also, from 1990 to 2010, the population of the previous decade is listed and we need to make sure we get the right one. For this task, we use regular expressions to select the column that contains the current year. Once we have found the right column, we rename it to `Population`. For 2019, we need to change it from `Population estimate, July 1, 2019[5]` to `Population`.

In [8]:
for year in range(1990,2020,10):
    d = {'Population as of{}.*'.format(year): 'Population'}
    Census[year].columns = Census[year].columns.to_series().replace(d, regex=True)
Census[2019].columns = Census[2019].columns.to_series().replace({'Population estimate, July 1, 2019.*': 'Population'}, regex=True)

Next, we get rid of all columns other than `State` and `Population`. We also remove all non-numerical characters from the `Population` column with some regular expression magic. Once we have done these two steps, we display the first 8 entries of all tables.

In [10]:
for key in Census.keys():
    print("Displaying values for {}".format(key))
    Census[key] = Census[key][['State','Population']]
    Census[key].loc[:,'Population']=Census[key].loc[:,'Population'].apply(
        lambda s: int(re.sub(r'\[.*','', str(s)).replace(',','')))
    display(Census[key].head(8))

Displaying values for 1790


,State,Population
0,Vermont,85539
1,New Hampshire,141885
2,Maine,96540
3,Massachusetts,378787
4,Rhode Island,68825
5,Connecticut,237946
6,New York,340120
7,New Jersey,184139


Displaying values for 1800


,State,Population
0,New Hampshire,183858
1,Massachusetts,422845
2,Maine,151719
3,Connecticut,251002
4,Vermont,154465
5,Rhode Island,69122
6,"New York (excluding Duchess, Ulster, Orange co...",484065
7,"New York (Duchess, Ulster, Orange counties)",101985


Displaying values for 1810


...

,State,Population
0,California,33871648
1,Texas,20851820
2,New York,18976457
3,Florida,15982378
4,Illinois,12419293
5,Pennsylvania,12281054
6,Ohio,11353140
7,Michigan,9938444


Displaying values for 2010


,State,Population
0,California,37253956
1,Texas,25145561
2,New York,19378102
3,Florida,18801310
4,Illinois,12830632
5,Pennsylvania,12702379
6,Ohio,11536504
7,Michigan,9883640


Displaying values for 2019


,State,Population
0,California,39512223
1,Texas,28995881
2,Florida,21477737
3,New York,19453561
4,Pennsylvania,12801989
5,Illinois,12671821
6,Ohio,11689100
7,Georgia,10617423


## The year 1800 😒

In [11]:
display(Census[1800].head(10))

,State,Population
0,New Hampshire,183858
1,Massachusetts,422845
2,Maine,151719
3,Connecticut,251002
4,Vermont,154465
5,Rhode Island,69122
6,"New York (excluding Duchess, Ulster, Orange co...",484065
7,"New York (Duchess, Ulster, Orange counties)",101985
8,New Jersey,211149
9,Pennsylvania (eastern district),327979


Unfortunately, the year 1800 doesn't play well with the other years and requires some additional cleaning up (that's why parsing is not recommended). We can see that instead of `States`, the 1800 census data has smaller subdivisions. To remedy this, we match our list of the 50+1 states against the district names and rename them to a single state. For example, our row with `New York (Duchess, Ulster, Orange counties)` will just become `New York`. Once we have renamed them, we group them by state and sum up the populations on all rows that share the same state. Just to be safe, we apply the same action to every census year. Grouping also has the nice side effect of setting the index to our column that we group by (`State`). Finally, we rename our `Population` column to the census year. 

In [13]:
States = pd.read_csv('States.csv')['States']
StatesList = States.tolist()

if isinstance(Census[1790].index[0], int):
    for key in Census.keys():
        for i, row in Census[key].iterrows():
            for State in StatesList:
                if State in row['State']:
                    Census[key].loc[i,'State'] = State  
        Census[key] = Census[key][Census[key]['State'].isin(StatesList)]
        Census[key] = Census[key].groupby(['State'],squeeze=True).sum()
        Census[key].rename(columns={'Population':str(key)},inplace=True)
display(Census[1800].head(14))

,1800
State,
Connecticut,251002
Delaware,64273
District of Columbia,323297
Georgia,162686
Indiana,5641
Kentucky,220959
Maine,151719
Maryland,4276
Massachusetts,422845


It worked! Great, now we are ready to put everything in one big table and make some final checks.


In [14]:
result_df = pd.DataFrame(States).set_index('States')
for key in Census.keys():
    result_df = result_df.add(Census[key],fill_value=0)
result_df.to_csv('State_Census_Historical.csv')
result_df.tail(5)

,1790,1800,1810,1820,1830,1840,1850,1860,1870,1880,...,1930,1940,1950,1960,1970,1980,1990,2000,2010,2019
Virginia,747610.0,880200.0,877683.0,938261.0,1044054.0,1025227.0,1119348.0,1596318.0,1225163.0,1542180.0,...,2421851.0,2677773.0,3262000.0,3978000.0,4648494.0,5346818.0,6187358.0,7078515.0,8001024.0,8535519.0
Washington,NaN,NaN,NaN,NaN,NaN,NaN,1201.0,11594.0,23955.0,98268.0,...,1563396.0,1736191.0,2386000.0,2860000.0,3409169.0,4132156.0,4866692.0,5894121.0,6724540.0,7614893.0
West Virginia,NaN,NaN,105469.0,136808.0,176924.0,224537.0,302313.0,NaN,442014.0,648936.0,...,1729205.0,1961974.0,2006000.0,1857000.0,1744237.0,1949644.0,1793477.0,1808344.0,1852994.0,1787147.0
Wisconsin,NaN,NaN,NaN,1444.0,3635.0,30945.0,305391.0,775881.0,1054670.0,1399750.0,...,2939006.0,3137587.0,3449000.0,3964000.0,4417933.0,4705767.0,4891769.0,5363675.0,5686986.0,5822434.0
Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9118.0,32610.0,...,225565.0,250742.0,292000.0,338000.0,332416.0,469557.0,453588.0,493782.0,563626.0,578759.0


It looks like some years have missing information because records were lost. For instance, there is no value for `West Virginia` in 1860. To fix this, we interpolate all missing values.

In [15]:
result_df.interpolate(axis=1, inplace = True)
result_df.tail()

,1790,1800,1810,1820,1830,1840,1850,1860,1870,1880,...,1930,1940,1950,1960,1970,1980,1990,2000,2010,2019
Virginia,747610.0,880200.0,877683.0,938261.0,1044054.0,1025227.0,1119348.0,1596318.0,1225163.0,1542180.0,...,2421851.0,2677773.0,3262000.0,3978000.0,4648494.0,5346818.0,6187358.0,7078515.0,8001024.0,8535519.0
Washington,NaN,NaN,NaN,NaN,NaN,NaN,1201.0,11594.0,23955.0,98268.0,...,1563396.0,1736191.0,2386000.0,2860000.0,3409169.0,4132156.0,4866692.0,5894121.0,6724540.0,7614893.0
West Virginia,NaN,NaN,105469.0,136808.0,176924.0,224537.0,302313.0,372163.5,442014.0,648936.0,...,1729205.0,1961974.0,2006000.0,1857000.0,1744237.0,1949644.0,1793477.0,1808344.0,1852994.0,1787147.0
Wisconsin,NaN,NaN,NaN,1444.0,3635.0,30945.0,305391.0,775881.0,1054670.0,1399750.0,...,2939006.0,3137587.0,3449000.0,3964000.0,4417933.0,4705767.0,4891769.0,5363675.0,5686986.0,5822434.0
Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9118.0,32610.0,...,225565.0,250742.0,292000.0,338000.0,332416.0,469557.0,453588.0,493782.0,563626.0,578759.0


We can see how `West Virginia`'s 1860 result was successfully interpolated between its neighbors. Now we are ready to go and visualize the census data over the past 230 years!

## Data Visualization
Getting and cleaning the data was by far the hardest part. Now that we got this out of the way, we can use a visualization tool such as Flourish to get our final **US Census Bar Chart Race** which is available at https://public.flourish.studio/visualisation/1322083.

Congrats on completing this tutorial and I hope it can help you in your next data science project!

In [16]:
HTML("<iframe src='https://public.flourish.studio/visualisation/1322083/embed' frameborder='0' scrolling='no' style='width:100%;height:600px;'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/1322083/?utm_source=embed&utm_campaign=visualisation/1322083' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>")

## Licensing 
This work is licensed under a [Creative Commons Attribution 4.0 International
License][cc-by]. This means that you are free to:

 * **Share** — copy and redistribute the material in any medium or format
 * **Adapt** — remix, transform, and build upon the material for any purpose, even commercially.

Under the following terms:

 * **Attribution** — a link to this [github repo](https://github.com/AntonMu/Census2020)!
 
[![CC BY 4.0][cc-by-image]][cc-by]

[cc-by]: http://creativecommons.org/licenses/by/4.0/
[cc-by-image]: https://i.creativecommons.org/l/by/4.0/88x31.png
[cc-by-shield]: https://img.shields.io/badge/License-CC%20BY%204.0-lightgrey.svg